# LLM 101.b PoC

Goal: Ask a question to a LLM via API.

# Installs

In [5]:
# !pip install -q --upgrade google-generativeai

In [6]:
# !pip install langchain-google-genai

In [4]:
# !pip install python-dotenv

# Library

In [27]:
import google.generativeai as genai
import os

from dotenv import load_dotenv
from IPython.display import display
from IPython.display import Markdown
import textwrap
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain import PromptTemplate
from langchain.chains import LLMChain
from langchain.chains import SimpleSequentialChain

In [2]:
def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

# Core

## Setting API key

In [1]:
# Save api key in env var
!echo -e 'GOOGLE_API_KEY=' > .env

In [4]:
load_dotenv()

True

In [5]:
genai.configure(api_key=os.environ.get("GOOGLE_API_KEY"))

## Using API directly

In [6]:
model = genai.GenerativeModel(model_name = "gemini-pro")
model

genai.GenerativeModel(
    model_name='models/gemini-pro',
    generation_config={},
    safety_settings={},
    tools=None,
    system_instruction=None,
)

In [7]:
prompt = [
    "What is the actresses and actors of the tv shiow The Big Bang Theory?",
]

In [8]:
response = model.generate_content(prompt)

In [9]:
to_markdown(response.text)

> **Actresses:**
> 
> * Kaley Cuoco - Penny
> * Melissa Rauch - Bernadette
> * Mayim Bialik - Amy
> * Kunal Nayyar - Raj
> 
> **Actors:**
> 
> * Johnny Galecki - Leonard
> * Jim Parsons - Sheldon
> * Simon Helberg - Howard

## Using API via LangChain

In [10]:
langchain_llm = ChatGoogleGenerativeAI(model="gemini-pro")

In [11]:
result = langchain_llm.invoke("What is the capital of Finland?")

In [12]:
to_markdown(result.content)

> Helsinki

## Chain with 2 steps

In [14]:
template = """
What is the actresses and actors of the tv shiow {tv_show}?
"""

prompt = PromptTemplate(
    input_variables=["tv_show"],
    template=template,
)
     

In [18]:
result = langchain_llm.invoke(prompt.format(tv_show="The Last Airbender"))
to_markdown(result.content)

> **Main Cast**
> 
> * Noah Ringer as Aang
> * Dev Patel as Prince Zuko
> * Nicola Peltz as Katara
> * Jackson Rathbone as Sokka
> * Aasif Mandvi as Uncle Iroh
> * Shaun Toub as Iroh II
> * Seychelle Gabriel as Princess Yue
> * Cliff Curtis as Fire Lord Ozai
> * Summer Bishil as Azula
> * Keong Sim as Gyatso
> 
> **Supporting Cast**
> 
> * Francis Guinan as Zhao
> * Jessica Andres as Suki
> * Dee Bradley Baker as Appa and Momo
> * Grey DeLisle as Azula (voice)
> * Dante Basco as Zuko (voice)
> * Mae Whitman as Katara (voice)
> * Jack DeSena as Sokka (voice)
> * Mark Hamill as Fire Lord Ozai (voice)
> * Mako Iwamatsu as Iroh (voice)

In [20]:
chain = LLMChain(llm=langchain_llm, prompt=prompt)

In [24]:
# Run the chain only specifying the input variable.
result = chain.run("The Last Airbender")
to_markdown(result)

> **Main Cast:**
> 
> * Noah Ringer as Aang
> * Dev Patel as Zuko
> * Nicola Peltz as Katara
> * Jackson Rathbone as Sokka
> * Shaun Toub as Iroh
> 
> **Supporting Cast:**
> 
> * Cliff Curtis as Fire Lord Ozai
> * Aasif Mandvi as Zhao
> * Summer Bishil as Azula
> * Francis Guinan as Gyatso
> * Seychelle Gabriel as Yue
> * Jessica Andres as Suki
> * Randall Duk Kim as Jeong Jeong
> * Keong Sim as Bumi
> * Shaun Sipos as Haru
> * Tamlyn Tomita as Ursa

In [30]:
# Define second prompt
prompt2 = PromptTemplate(
    input_variables=["cast"],
    template="Tell me a fun fact of one of these actresses and actors: {cast}",
)
chain2 = LLMChain(llm=langchain_llm, prompt=prompt2)

In [31]:
overall_chain = SimpleSequentialChain(chains=[chain, chain2], verbose=True)


In [33]:
# Run the chain specifying only the input variable for the first chain.
explanation = overall_chain.run("Bridgerton")
print(explanation)



> Entering new SimpleSequentialChain chain...
**Main Cast:**

* Phoebe Dynevor as Daphne Bridgerton
* Regé-Jean Page as Simon Basset, Duke of Hastings
* Jonathan Bailey as Anthony Bridgerton, Viscount Bridgerton
* Nicola Coughlan as Penelope Featherington
* Claudia Jessie as Eloise Bridgerton
* Luke Thompson as Benedict Bridgerton
* Luke Newton as Colin Bridgerton
* Ruth Gemmell as Violet Bridgerton
* Adjoa Andoh as Lady Danbury
* Golda Rosheuvel as Queen Charlotte
* Polly Walker as Lady Portia Featherington
* Bessie Carter as Prudence Featherington
* Harriet Cains as Philipa Featherington
* Ben Miller as Archibald Featherington

**Recurring Cast:**

* Sabrina Bartlett as Siena Rosso
* Martins Imhangbe as Will Mondrich
* Ruby Stokes as Francesca Bridgerton
* Will Tilston as Gregory Bridgerton
* Florence Hunt as Hyacinth Bridgerton
* Jason Barnett as Jack
* Patricia Hodge as Lady Sheffield
* Simon Ludders as Lord Sheffield
* Hugo Nanette as Lord Fife
* Jessica Madsen as Cressida Cowpe

---
# End.